# Creates a list of bricks and their positions necessary to create words or phrases

* This can create multiple lines of bricks if you seperate them with the traditional '\n'.

Execute the below code blocks then put your own text in the block that currently has 
```Python
GetBricksFromText("Tesla\nP100D")
```
The resulting json can be used in the next step to render the bricks to a .stl (see README)

In [1]:
import pandas as pd
import numpy as np
from bitstring import BitArray, BitStream

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Install a conda package in the current Jupyter kernel
# ONLY run this if you do not yet have pandas and bitstring
import sys
!conda install --yes --prefix {sys.prefix} pandas
!{sys.executable} -m pip install bitstring

In [2]:
%config IPCompleter.greedy=True

# ASCII chart for 5x7 bitmap
## From [font5x7.h](https://github.com/adafruit/monochron/blob/master/firmware/font5x7.h) 

The ascii chart below is closely derived from the font5x7.h originally produced by Pascal Stand on 10/19/2001.  The header below was from the original file.
```c
/*! \file font5x7.h \brief Graphic LCD Font (Ascii Characters). */
//*****************************************************************************
//
// File Name	: 'font5x7.h'
// Title		: Graphic LCD Font (Ascii Charaters)
// Author		: Pascal Stang
// Date			: 10/19/2001
// Revised		: 10/19/2001
// Version		: 0.1
// Target MCU	: Atmel AVR
// Editor Tabs	: 4
//
//*****************************************************************************
```

In [3]:
ascii = \
   {' ': [0x00, 0x00, 0x00, 0x00, 0x00],
	'!': [0x00, 0x00, 0x5F, 0x00, 0x00],
	'"': [0x00, 0x07, 0x00, 0x07, 0x00],
	'#': [0x14, 0x7F, 0x14, 0x7F, 0x14],
	'$': [0x24, 0x2A, 0x7F, 0x2A, 0x12],
	'%': [0x23, 0x13, 0x08, 0x64, 0x62],
	'&': [0x36, 0x49, 0x55, 0x22, 0x50],
	'\'': [0x00, 0x05, 0x03, 0x00, 0x00],  
	'(': [0x00, 0x1C, 0x22, 0x41, 0x00],
	')': [0x00, 0x41, 0x22, 0x1C, 0x00],
	'*': [0x08, 0x2A, 0x1C, 0x2A, 0x08],
	'+': [0x08, 0x08, 0x3E, 0x08, 0x08],
	',': [0x00, 0x50, 0x30, 0x00, 0x00],
	'-': [0x08, 0x08, 0x08, 0x08, 0x08],
	'.': [0x00, 0x60, 0x60, 0x00, 0x00],
	'/': [0x20, 0x10, 0x08, 0x04, 0x02],
	'0': [0x3E, 0x51, 0x49, 0x45, 0x3E],
	'1': [0x00, 0x42, 0x7F, 0x40, 0x00],
	'2': [0x42, 0x61, 0x51, 0x49, 0x46],
	'3': [0x21, 0x41, 0x45, 0x4B, 0x31],
	'4': [0x18, 0x14, 0x12, 0x7F, 0x10],
	'5': [0x27, 0x45, 0x45, 0x45, 0x39],
	'6': [0x3C, 0x4A, 0x49, 0x49, 0x30],
	'7': [0x01, 0x71, 0x09, 0x05, 0x03],
	'8': [0x36, 0x49, 0x49, 0x49, 0x36],
	'9': [0x06, 0x49, 0x49, 0x29, 0x1E],
	':': [0x00, 0x36, 0x36, 0x00, 0x00],
	';': [0x00, 0x56, 0x36, 0x00, 0x00],
	'<': [0x00, 0x08, 0x14, 0x22, 0x41],
	'=': [0x14, 0x14, 0x14, 0x14, 0x14],
	'>': [0x41, 0x22, 0x14, 0x08, 0x00],
	'?': [0x02, 0x01, 0x51, 0x09, 0x06],
	'@': [0x32, 0x49, 0x79, 0x41, 0x3E],
	'A': [0x7E, 0x11, 0x11, 0x11, 0x7E],
	'B': [0x7F, 0x49, 0x49, 0x49, 0x36],
	'C': [0x3E, 0x41, 0x41, 0x41, 0x22],
	'D': [0x7F, 0x41, 0x41, 0x22, 0x1C],
	'E': [0x7F, 0x49, 0x49, 0x49, 0x41],
	'F': [0x7F, 0x09, 0x09, 0x01, 0x01],
	'G': [0x3E, 0x41, 0x41, 0x51, 0x32],
	'H': [0x7F, 0x08, 0x08, 0x08, 0x7F],
	'I': [0x00, 0x41, 0x7F, 0x41, 0x00],
	'J': [0x20, 0x40, 0x41, 0x3F, 0x01],
	'K': [0x7F, 0x08, 0x14, 0x22, 0x41],
	'L': [0x7F, 0x40, 0x40, 0x40, 0x40],
	'M': [0x7F, 0x02, 0x04, 0x02, 0x7F],
	'N': [0x7F, 0x04, 0x08, 0x10, 0x7F],
	'O': [0x3E, 0x41, 0x41, 0x41, 0x3E],
	'P': [0x7F, 0x09, 0x09, 0x09, 0x06],
	'Q': [0x3E, 0x41, 0x51, 0x21, 0x5E],
	'R': [0x7F, 0x09, 0x19, 0x29, 0x46],
	'S': [0x46, 0x49, 0x49, 0x49, 0x31],
	'T': [0x01, 0x01, 0x7F, 0x01, 0x01],
	'U': [0x3F, 0x40, 0x40, 0x40, 0x3F],
	'V': [0x1F, 0x20, 0x40, 0x20, 0x1F],
	'W': [0x7F, 0x20, 0x18, 0x20, 0x7F],
	'X': [0x63, 0x14, 0x08, 0x14, 0x63],
	'Y': [0x03, 0x04, 0x78, 0x04, 0x03],
	'Z': [0x61, 0x51, 0x49, 0x45, 0x43],
	'[': [0x00, 0x00, 0x7F, 0x41, 0x41],
	'"': [0x02, 0x04, 0x08, 0x10, 0x20],
	']': [0x41, 0x41, 0x7F, 0x00, 0x00],
	'^': [0x04, 0x02, 0x01, 0x02, 0x04],
	'_': [0x40, 0x40, 0x40, 0x40, 0x40],
	'`': [0x00, 0x01, 0x02, 0x04, 0x00],
	'a': [0x20, 0x54, 0x54, 0x54, 0x78],
	'b': [0x7F, 0x48, 0x44, 0x44, 0x38],
	'c': [0x38, 0x44, 0x44, 0x44, 0x20],
	'd': [0x38, 0x44, 0x44, 0x48, 0x7F],
	'e': [0x38, 0x54, 0x54, 0x54, 0x18],
	'f': [0x08, 0x7E, 0x09, 0x01, 0x02],
	'g': [0x08, 0x14, 0x54, 0x54, 0x3C],
	'h': [0x7F, 0x08, 0x04, 0x04, 0x78],
	'i': [0x00, 0x44, 0x7D, 0x40, 0x00],
	'j': [0x20, 0x40, 0x44, 0x3D, 0x00],
	'k': [0x00, 0x7F, 0x10, 0x28, 0x44],
	'l': [0x00, 0x41, 0x7F, 0x40, 0x00],
	'm': [0x7C, 0x04, 0x18, 0x04, 0x78],
	'n': [0x7C, 0x08, 0x04, 0x04, 0x78],
	'o': [0x38, 0x44, 0x44, 0x44, 0x38],
	'p': [0x7C, 0x14, 0x14, 0x14, 0x08],
	'q': [0x08, 0x14, 0x14, 0x18, 0x7C],
	'r': [0x7C, 0x08, 0x04, 0x04, 0x08],
	's': [0x48, 0x54, 0x54, 0x54, 0x20],
	't': [0x04, 0x3F, 0x44, 0x40, 0x20],
	'u': [0x3C, 0x40, 0x40, 0x20, 0x7C],
	'v': [0x1C, 0x20, 0x40, 0x20, 0x1C],
	'w': [0x3C, 0x40, 0x30, 0x40, 0x3C],
	'x': [0x44, 0x28, 0x10, 0x28, 0x44],
	'y': [0x0C, 0x50, 0x50, 0x50, 0x3C],
	'z': [0x44, 0x64, 0x54, 0x4C, 0x44],
	'{': [0x00, 0x08, 0x36, 0x41, 0x00],
	'|': [0x00, 0x00, 0x7F, 0x00, 0x00],
	'}': [0x00, 0x41, 0x36, 0x08, 0x00]}

In [4]:
def asciiToBits(letter):
    tmp = [[0]*5 for i in range(7)]
    for column in range(5): 
        bits = BitArray('uint:7=' + str(ascii[letter][column])).bin
        for row in range(7):
            tmp[row][column] = int(bits[6-row])
    return tmp.copy()

In [5]:
def letterToDF(letter):
    df = pd.DataFrame(asciiToBits(letter))
    #df.columns = [letter]*5
    #df[df == '0'] = ' '
    return df

In [6]:
letterToDF('8')

,0,1,2,3,4
0,0,1,1,1,0
1,1,0,0,0,1
2,1,0,0,0,1
3,0,1,1,1,0
4,1,0,0,0,1
5,1,0,0,0,1
6,0,1,1,1,0


In [7]:
def stringToDF(string):
    tmp = []
    first = True
    df_between = pd.DataFrame({' ': [0, 0, 0, 0, 0, 0, 0]})
    for letter in string:
        if first:
            first = False
        else:
            tmp.append(df_between)
        tmp.append(letterToDF(letter))
    joined = pd.concat(tmp, axis=1, sort=False)
    joined.columns = [np.arange(0,joined.shape[1])]
    return joined
    #return tmp

In [8]:
stringToDF('Hi')

,0,1,2,3,4,5,6,7,8,9,10
0,1,0,0,0,1,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,1,0,0
3,1,1,1,1,1,0,0,0,1,0,0
4,1,0,0,0,1,0,0,0,1,0,0
5,1,0,0,0,1,0,0,0,1,0,0
6,1,0,0,0,1,0,0,1,1,1,0


In [9]:
brickColumns = ['StartRow', 'EndRow', 'StartColumn', 'EndColumn', 'BrickSize']
bricks = pd.DataFrame(columns=brickColumns)
df_tmp = pd.DataFrame()
coll=-1

def findBrick(dat,transformed):
    global coll
    global bricks
    global brickColumns
    global df_tmp
    
    if (type(dat.index[0]) == np.int64):
        firstRow = dat.index[0]
        lastRow = dat.index[dat.index.shape[0]-1]
    else:
        firstRow = dat.index[0][0]
        lastRow = dat.index[dat.index.shape[0]-1][0]
    if firstRow==0:
        coll = coll + 1  
    
    if transformed==False:  
        if (dat==[1]*dat.index.shape[0]).all():
            print(f'   match at [{firstRow}-{lastRow},{coll}]\n')
            newbrick = pd.DataFrame([[firstRow,lastRow,coll,coll,dat.index.shape[0]]], columns = brickColumns)
            bricks = bricks.append(newbrick)
            df_tmp.at[dat.index,coll] = 0
            return 1
        else:
            return 0
    else:
        if (dat==[1]*dat.index.shape[0]).all():
            print(f'   match at [{coll},{firstRow}-{lastRow}]\n')
            newbrick = pd.DataFrame([[coll,coll,firstRow,lastRow,dat.index.shape[0]]], columns = brickColumns)
            bricks = bricks.append(newbrick)
            df_tmp.at[dat.index.labels[0],coll] = 0
            return 1
        else:
            return 0

    
def findBricks(df):
    rows = df.shape[0]
    columns = df.shape[1]
    global coll
    global df_tmp
    global bricks
    df_tmp = df.copy()
    bricks = pd.DataFrame(columns=brickColumns)
    
    df_pretty = df.copy()
    df_pretty[df_pretty==0] = ' '
    print(df_pretty)
    
    print('Looking for horizontal bricks')
    
    # look horizontal first
    df_tmp = df_tmp.transpose() 
    for i in reversed(range(2,columns+1)):
        #print(f'Processing for window = {i}')
        coll=-1
        df_tmp.rolling(window=i,axis=0).apply(findBrick,raw=False, kwargs={'transformed':True})
    
    print('Looking for vertical bricks')
    
    # translate back to look at vertical bars left
    df_tmp = df_tmp.transpose()
    for i in reversed(range(1,rows+1)):
        coll=-1
        df_tmp.rolling(window=i,axis=0).apply(findBrick,raw=False, kwargs={'transformed':False})
        
    bricks.index = [np.arange(0,bricks.shape[0])]

    print('DataFrame that lists the bricks to construct the word(s):\n\n')
    
    return bricks

In [10]:
def GetSpacedLinesFromText(text):
    lines = text.splitlines()
    most_characters = 0

    filled_lines = []

    for line in lines:
        if (most_characters < len(line)):
            most_characters = len(line)

    for line in lines:
        spacing = most_characters - len(line)
        filled_lines.append(line + ' '*spacing) 

    return filled_lines

In [11]:
def GetBricksFromText(text):
    blank_line = 0
    lines = GetSpacedLinesFromText(text)
    first = True;
    ret = 0
    
    for line in lines:
        if first:
            first = False
            ret = stringToDF(line)
            blank_line = pd.DataFrame([[0] * len(ret.columns)], columns=ret.columns)
        else:
            ret = ret.append(blank_line)
            ret = ret.append(stringToDF(line))
    ret.index = [np.arange(0,ret.shape[0])]
    bricks = findBricks(ret)
    
    return bricks.to_json(orient='index')

In [12]:
bricksJson = GetBricksFromText("Tesla\nP100D")

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
0   1  1  1  1  1                                            1  1                        
1         1                                                     1                        
2         1              1  1  1           1  1  1              1              1  1  1   
3         1           1           1     1                       1                       1
4         1           1  1  1  1  1        1  1  1              1              1  1  1  1
5         1           1                             1           1           1           1
6         1              1  1  1        1  1  1  1           1  1  1           1  1  1  1
7                                                                                        
8   1  1  1  1              1              1  1  1           1  1  1        1  1  1      
9   1           1        1  1           1           1     1           1     1        1   
10  1     

Merge brick_maker.js with the json of the bricks.

In [ ]:
file = open('bricks_openjscad.js', 'w')
file2 = open('brick_maker.js', 'r')
file.write('var bricks = ')
file.write(bricksJson) 
file.write(';')
file.write(file2.read())
file.close()